<a href="https://colab.research.google.com/github/SophieShin/NLP_22_Fall/blob/main/%5BSSH%5D_assign1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1: Generating Word Embeddings
- Please save a copy to your drive before working on the notebook.
- Save your work as $<$initials$>$-assign1.ipynb, e.g. `RCE-assign1.ipynb`
- Make sure your notebook is accessible (readable) to anyone with the link and provide the url when submitting.
- You may also submit your ipynb file if you prefer.

# Word Embeddings 
Word embeddings are dense vectors of real numbers, one per word in your vocabulary. We often want dense outputs from our neural networks, where the inputs are $|V|$ dimensional, where $𝑉$ is our vocabulary, but often the outputs are only a few dimensional (if we are only predicting a handful of labels, for instance). How do we get from a massive dimensional space to a smaller dimensional space?

When we use a one-hot encoding,
we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will have a 1 in some other location, and a 0 everywhere else. It basically treats all words as independent entities with no relation to each other. What we really want is some notion of *similarity* between words.

How could we actually encode semantic similarity in words? Maybe we think up some semantic attributes. 

For example, in these sentences:
* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

We see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to run" semantic attribute.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same direction) will have similarity 1. Extremely dissimilar words should have similarity -1.

## Word Embeddings in Pytorch

Embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned index $i$ has its embedding stored in the $i$'th row of the matrix.

## An Example: N-Gram Language Modelling

A language model is a system that predicts a word when other words in a sequence are given, e.g. "Where is my wonderful _". In an $n$-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the $i$-th word of the sequence.

Since the sequence length can be long, a very crude way of doing this is to restrict the number of previous consecutive words that will be considered when predicting $w_i$. This number of $n$ consecutive words is called an **$n$-gram**. The 2-grams for the sequence "Where is my wonderful" are "Where is", "is my" and "my wonderful".

The algorithm scans each word one by one, each time trying to predict the word at the current index, $w_i$, based on a few previous context words. The number of context words to consider is controlled by the hyper parameter, `CONTEXT_SIZE`, which is the same as $n-1$. The prediction produced by the model is then compared to the actual word at position $i$ using a loss function.

In this example, we will compute the loss function on the text from an online book and update the parameters with backpropagation.


### Download James Joyce's book from Gutenberg

---


In [23]:
# Q1. Download James Joyce's 'A Picture of Dorian Gray' and save it as book.txt
# Download URL:
!wget https://www.gutenberg.org/cache/epub/4078/pg4078.txt -O book.txt
# CODE

--2022-09-24 05:46:18--  https://www.gutenberg.org/cache/epub/4078/pg4078.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 324893 (317K) [text/plain]
Saving to: ‘book.txt’

book.txt            100%[===================>] 317.28K  --.-KB/s    in 0.1s    

2022-09-24 05:46:18 (3.04 MB/s) - ‘book.txt’ saved [324893/324893]



Open and Inspect File

In [24]:
import pandas as pd

# Open this file for reading and store contents in a string
with open('book.txt', 'r') as f:
    corpus_ = f.read()

# See some text in this document
print(corpus_[1075:3075])

# Select a subset to build our vocab
corpus_ = corpus_[1075:50750]

# Convert the String object to a list
corpus = [corpus_]


 The studio was filled with the rich odor of roses, and when the
light summer wind stirred amidst the trees of the garden there came
through the open door the heavy scent of the lilac, or the more
delicate perfume of the pink-flowering thorn.

From the corner of the divan of Persian saddle-bags on which he was
lying, smoking, as usual, innumerable cigarettes, Lord Henry Wotton
could just catch the gleam of the honey-sweet and honey-colored
blossoms of the laburnum, whose tremulous branches seemed hardly able
to bear the burden of a beauty so flame-like as theirs; and now and
then the fantastic shadows of birds in flight flitted across the long
tussore-silk curtains that were stretched in front of the huge window,
producing a kind of momentary Japanese effect, and making him think of
those pallid jade-faced painters who, in an art that is necessarily
immobile, seek to convey the sense of swiftness and motion.  The sullen
murmur of the bees shouldering their way through the long unmown g

### Tokenise and Build Vocabulary

In [25]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens():
  for s in corpus:
    tokens = tokenizer(s)
    print(tokens)
    yield tokens

token_generator = yield_tokens() # can now iterate through token_generator



In [26]:
token_generator = yield_tokens() # can now iterate through token_generator

# Q2. Build a vocab using build_vocab_from_iterator():
# - Out of vocab (OOV) words should be given the token "<unk>"
# - Select only words that occur at least 2 times in the document using the 'min_freq' argument

vocab = build_vocab_from_iterator(token_generator, specials=["<unk>"], min_freq=2)
vocab.set_default_index(vocab["<unk>"]) # to assign unknown token with index 0



['the', 'studio', 'was', 'filled', 'with', 'the', 'rich', 'odor', 'of', 'roses', ',', 'and', 'when', 'the', 'light', 'summer', 'wind', 'stirred', 'amidst', 'the', 'trees', 'of', 'the', 'garden', 'there', 'came', 'through', 'the', 'open', 'door', 'the', 'heavy', 'scent', 'of', 'the', 'lilac', ',', 'or', 'the', 'more', 'delicate', 'perfume', 'of', 'the', 'pink-flowering', 'thorn', '.', 'from', 'the', 'corner', 'of', 'the', 'divan', 'of', 'persian', 'saddle-bags', 'on', 'which', 'he', 'was', 'lying', ',', 'smoking', ',', 'as', 'usual', ',', 'innumerable', 'cigarettes', ',', 'lord', 'henry', 'wotton', 'could', 'just', 'catch', 'the', 'gleam', 'of', 'the', 'honey-sweet', 'and', 'honey-colored', 'blossoms', 'of', 'the', 'laburnum', ',', 'whose', 'tremulous', 'branches', 'seemed', 'hardly', 'able', 'to', 'bear', 'the', 'burden', 'of', 'a', 'beauty', 'so', 'flame-like', 'as', 'theirs', 'and', 'now', 'and', 'then', 'the', 'fantastic', 'shadows', 'of', 'birds', 'in', 'flight', 'flitted', 'across

In [27]:
# Q3. How many words are in this vocab?
print(f'Theare are {len(vocab)} words in the vocab')

# Q4. Get the word at index 120 in vocab
print(vocab.lookup_tokens([120]))


# Q5. Get the index for the word "thorn". What does this tell you about the occurrence of this word in the document?
print(vocab.lookup_indices(['thorn']))
# A5. Since the index is 0(unknown token), we can infer that this word appeared less than twice. 

Theare are 738 words in the vocab
['look']
[0]


### Define the Neural Network & Train
- Choose a context size, `CONTEXT_SIZE` i.e. the number of previous words you want to consider for the prediction
- Choose an embedding dimension, `EMBEDDING_DIM`
- Get the $n$-grams based on the context size
- Feed the $n$-grams as input and predict the next word using the weights (embeddings) which are random at the beginning
- Calculate loss and backpropagate to update the weights

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

CONTEXT_SIZE = 5
EMBEDDING_DIM = 12
EPOCHS = 100

train_corpus = corpus_.split()

# Generate the n-grams
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [train_corpus[i - j - 1] for j in reversed(range(CONTEXT_SIZE))],
        train_corpus[i]
    )
    for i in range(CONTEXT_SIZE, len(train_corpus))
]

# Print the first 3, just so you can see what they look like.
print(f"First 3 n-grams and target word: {ngrams[:3]}")


class NGramLM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLM, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


loss_function = nn.NLLLoss()
model = NGramLM(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(EPOCHS):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([vocab[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([vocab[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    if ((epoch+1) % 5 == 0):
      print(f"Epoch {epoch+1} Loss: {total_loss}")  # Print loss every 5 epochs


First 3 n-grams and target word: [(['The', 'studio', 'was', 'filled', 'with'], 'the'), (['studio', 'was', 'filled', 'with', 'the'], 'rich'), (['was', 'filled', 'with', 'the', 'rich'], 'odor')]
Epoch 5 Loss: 30456.737997146323
Epoch 10 Loss: 22989.707252198248
Epoch 15 Loss: 17439.238761334986
Epoch 20 Loss: 13753.080832084022
Epoch 25 Loss: 10976.114565746577
Epoch 30 Loss: 9152.739669792747
Epoch 35 Loss: 7904.439418785955
Epoch 40 Loss: 7149.287117404321
Epoch 45 Loss: 6465.860707775359
Epoch 50 Loss: 5776.90755209173
Epoch 55 Loss: 5729.843036224383
Epoch 60 Loss: 5437.030284629293
Epoch 65 Loss: 5390.077414384929
Epoch 70 Loss: 5534.161462508613
Epoch 75 Loss: 4742.506790247099
Epoch 80 Loss: 4602.237487769577
Epoch 85 Loss: 5293.983554475634
Epoch 90 Loss: 5404.820579453093
Epoch 95 Loss: 5499.58792296397
Epoch 100 Loss: 5375.184724859636


In [ ]:
# Q6. How many units (neurons) are there in the final layer and why?
# A6. It is the size of the vocab(738) therefore we can apply softmax to make the model predict the most possible word.

# Q7. What does this line of code do?
# embeds = self.embeddings(inputs).view((1, -1))
# A7. This flattens the embedding tensor to a vector.

### See how well the model predicts the words in the document

In [31]:
# Check accuracy for the first 25 words
num_correct = 0
for i in range(25):
  word, target = ngrams[i]
  word = torch.tensor([vocab[i] for i in word], dtype=torch.long)
  out = model(word)
  _, predict_label = torch.max(out, 1)
  itos = vocab.get_itos()
  predict_word = itos[predict_label.item()]
  print(f'Real word is "{target}"", Predicted word is "{predict_word}"')
  if target==predict_word:
    num_correct += 1 
print(f"Correctly predicted words = {num_correct}/25")

Real word is "the"", Predicted word is "the"
Real word is "rich"", Predicted word is "<unk>"
Real word is "odor"", Predicted word is "<unk>"
Real word is "of"", Predicted word is "of"
Real word is "roses,"", Predicted word is "<unk>"
Real word is "and"", Predicted word is "and"
Real word is "when"", Predicted word is "when"
Real word is "the"", Predicted word is "he"
Real word is "light"", Predicted word is "light"
Real word is "summer"", Predicted word is "summer"
Real word is "wind"", Predicted word is "wind"
Real word is "stirred"", Predicted word is "stirred"
Real word is "amidst"", Predicted word is "<unk>"
Real word is "the"", Predicted word is "the"
Real word is "trees"", Predicted word is "trees"
Real word is "of"", Predicted word is "of"
Real word is "the"", Predicted word is "the"
Real word is "garden"", Predicted word is "garden"
Real word is "there"", Predicted word is "there"
Real word is "came"", Predicted word is "came"
Real word is "through"", Predicted word is "through

### Model improvement
The accuracy was probably not great. Using the options given below, see if you can achieve 50% accuracy.
- Notes:
  - Training takes 3.5 minutes with the settings above (if epochs are doubled, it would double the training time)
  - If a different optimiser is used, training could take longer

In [ ]:
# Q8. You should try to get the at least average 12.5/25 accuracy over 3 runs. 
# Tweak the following hyper parameters in order to achieve this:
# i) CONTEXT_SIZE = (between 3 and 5), ii) EPOCHS = (max 100), iii) EMBEDDING_DIM = (between 8 and 12), iv) optimizer (any other)
# Train with the same configuration 3 times to see that the (average) accuracy is at least 50% (12.5/25).
# Your final code in the cells above should reflect the model that has achieved this accuracy.
# All code cell outputs above should be clear. You may duplicate the training and prediction code cells and run them 
# but it is not necessary. One run's output is enough. You should jot down the three runs accuracies below.
# State any comments or observations that you have with regards to this.

# Run 1 accuracy: 6/25  -> epochs:100
# Run 2 accuracy: 12/25   -> epochs:100, context_size: 5
# Run 3 accuracy: 20/25   -> epochs:100, context_size = 5, embedding_dim = 12 , learning_rate : 0.01

### Get embedding for a word

In [101]:
vocab['trees']

712

In [72]:
# Q9. Get the embedding for a given word. It should return the embedding generated by the model or a 1D tensor containing
# EMBEDDING_DIM 0s if the word is not in the vocabulary 
# CODE
word_dict = {'trees':vocab["trees"]}
index=[word_dict['trees']]
input_tensor=torch.LongTensor(index)
emb_layer=model.embeddings
embeddings=emb_layer(input_tensor)
print(embeddings)

tensor([[-0.2733,  1.8189,  0.1303, -0.7887, -1.1634,  2.0401,  1.1422, -0.6723,
         -0.4153,  0.5764, -1.5664, -0.1830]], grad_fn=<EmbeddingBackward0>)


In [103]:
print(vocab['skku'])
word_dict = {'skku':vocab["skku"]}
index=[word_dict['skku']]
input_tensor=torch.LongTensor(index)
emb_layer=model.embeddings
embeddings=emb_layer(input_tensor)
print(embeddings)

0
tensor([[-0.2540, -0.3412,  0.3125,  0.7686, -0.8240, -0.6898,  0.8095, -0.4827,
          0.0261,  0.2834, -0.3079, -0.5027]], grad_fn=<EmbeddingBackward0>)


### Find "similar" words
Based on the embeddings, we hope that vectors that are similar correspond to words that are similar in the context of the document

In [91]:
def closest_to_vector(v, n):
    embeddings = model.embeddings.weight

    all_scores = torch.matmul(embeddings, v)
    best_inds = reversed(torch.argsort(all_scores))   # why reversed?
    itos = vocab.get_itos()
    best_words = [itos[i] for i in best_inds]
    return best_words[:n]

def most_similar(w, n):
    w_emb = model.embeddings.weight[vocab[w]]
    return closest_to_vector(model.embeddings.weight[vocab[w], :], n)


In [73]:
embeddings = model.embeddings.weight

In [92]:
embeddings.shape

torch.Size([738, 12])

In [76]:
model.embeddings.weight[vocab['portrait'],:]

tensor([-1.8624,  0.0027,  1.4693, -0.6728,  0.4088, -0.5114,  0.7859,  1.4218,
        -0.2440, -1.3600, -2.2400, -0.6394], grad_fn=<SliceBackward0>)

In [80]:
v = model.embeddings.weight[vocab['portrait']]
v

tensor([-1.8624,  0.0027,  1.4693, -0.6728,  0.4088, -0.5114,  0.7859,  1.4218,
        -0.2440, -1.3600, -2.2400, -0.6394], grad_fn=<SelectBackward0>)

In [83]:
all_scores = torch.matmul(embeddings, v)
all_scores.shape


torch.Size([738])

In [ ]:
all_scores

In [84]:
best_inds = reversed(torch.argsort(all_scores))
best_inds

tensor([179, 662, 697, 705, 528, 684, 303, 373, 256, 358,  48, 134, 209, 604,
        386, 435, 131, 108, 655,  58, 272, 585,  85, 213, 452, 548, 625, 565,
        141,  72, 651, 707, 703, 474, 341,  75, 397, 526, 560, 115, 507,  68,
         14,  82, 483,  57, 706, 391, 590, 492, 103, 681, 117, 189, 453, 154,
         23, 291, 471, 680,  18, 514, 591, 400, 454, 255, 305, 525, 399, 339,
        649,  28, 282,  76, 317, 718, 346, 200, 126, 184, 674, 736, 278, 598,
        328, 211,  80, 694, 173, 575, 436, 381, 465, 729, 145, 193, 298, 228,
        461, 529, 517, 481, 735,  16, 556, 261, 578, 539, 521, 570, 246, 352,
        631, 545, 518, 722, 264,  38, 237, 111,  12,  95, 425, 469, 607, 275,
        475, 727,  59, 457, 268, 227, 277, 618, 434, 343, 395, 644, 690, 210,
        669, 666,  87, 247, 449, 349, 205, 606,  94, 608,  78, 489,  20, 426,
        659, 511, 208, 658, 555,   7, 613, 572, 414, 285, 444, 318, 557, 244,
        617,  89, 497, 182, 367, 665, 689, 512, 342, 480, 176, 5

In [90]:
itos = vocab.get_itos()
best_words = [itos[i] for i in best_inds]
best_words[:10]

['portrait',
 'remember',
 'sunlight',
 'thoughts',
 'colors',
 'smiling',
 'exhibit',
 'became',
 'friendship',
 'to-day']

Get the 10 most similar words to the word "portrait"

In [70]:
print(most_similar('portrait', 10))

['portrait', 'remember', 'sunlight', 'thoughts', 'colors', 'smiling', 'exhibit', 'became', 'friendship', 'to-day']


### Further Thoughts

In [ ]:
# Q10. Do you find the top-10 most similar words convincing?
# A10. It seems to show similar words to a cetain extent. 
# What would you do to further improve the performance of the model? You can suggest anything apart from the list of hyperparameters
# in Q8. You do NOT have to implement anything.
# Putting some linear layers in the model may increase the performance.
